In [124]:
from math import log
import nltk

Arthur = open('books/Arthur.txt').read()
Jane = open('books/Jane.txt').read()
files_dict = {"Arthur":Arthur, "Jane":Jane}

In [125]:
cat_word_dict = {}       # word frequence of each class
cat_word_count_dict = {} # number of words of each class

for key in files_dict:   # format {"Arthur":Arthur texts, "Jane":Jane texts}
    list_words = nltk.word_tokenize(files_dict[key].decode('utf-8'))
    cat_word_dict[key] = cat_word_dict.get(key,{})
    cat_word_count_dict[key] = cat_word_count_dict.get(key,0) # number of words of this class
    cat_word_count_dict[key] += len(list_words)

    for w in list_words:
        cat_word_dict[key][w] = cat_word_dict[key].get(w, 0) # count word frequence of each class
        cat_word_dict[key][w]+=1
    
# count how many unique words in whole dataset, which will be used in calculating conditional probability
num_vocabulary = 0            
for word in cat_word_dict.values():
     num_vocabulary += len(word)

In [126]:
# Multinomial Naive Bayes model, input is a piece of text and return the category
def MultinomialNB(text):
    min_neg_log_prob=1000000000 # maximize log () equals to minimize - log ()
    min_cat='' # the cat we are choosen
    list_words = nltk.word_tokenize(text.decode('utf-8')) # spilit raw text to words

    for cat in  cat_word_count_dict:
        neg_log_prob = - log( cat_word_count_dict[cat] * 1.0 / sum(cat_word_count_dict.values())) # class prior prob, consider number of words because we only have two training documents
        word_dict = cat_word_dict[cat]
        count_cat = cat_word_count_dict[cat]
        for w in list_words:
            count_word_train = word_dict.get(w,0)
            cond_prob = (count_word_train + 1.0) / (count_cat + num_vocabulary)
            neg_log_prob -= log(cond_prob)     
        if min_neg_log_prob > neg_log_prob:
            min_cat = cat
            min_neg_log_prob = neg_log_prob
    return min_cat


In [127]:
#  Process two files into testing set with many divided paragraphs
texts, labels = [], []
for i, line in enumerate(Arthur.split("\r\n\r\n")):
    line = line.replace('\r\n', '')
    if line is not '':        
        labels.append('Arthur')
        texts.append(line)
for i, line in enumerate(Jane.split("\r\n\r\n")):
    line = line.replace('\r\n', '')
    if line is not '':
        labels.append('Jane')
        texts.append(line)        

In [128]:
## testing
i = 0 # count the number of correct prediction
for id in range(len(texts)):
    pred = MultinomialNB(texts[id])
    i += int(pred == labels[id])
print i * 1.0 / len(texts)  # the accuracy is 0.950280264978

0.950280264978
